<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Templates/tg/tg_bot_template_async.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install aiogram

#Async approach

## Imports

In [10]:
import cv2
import asyncio
import logging
import numpy as np

from aiogram import F
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram.types import FSInputFile, BufferedInputFile, Message
from google.colab import userdata
from PIL import Image, ImageDraw

## init bot and async

In [3]:
TOKEN = userdata.get('trybot')

In [4]:
iobot = Bot(token=TOKEN)
dp = Dispatcher()
user_history = {}

### Set up logging

In [5]:
# Set up logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
def log_user(message, photopath, result):
  user_id = message.from_user.id
  if user_id not in user_history:
      user_history[user_id] = []

  user_history[user_id].append({
        'message_id': message.message_id,
        'text': message.text,
        'photo_path': photopath,
        'result': result})

## Handle messages

In [7]:
@dp.message(Command('start'))
async def handle_start(message):
    await message.answer("Welcome! Send me a photo, and I'll process it.")

@dp.message(Command('help'))
async def handle_help(message):
    # Provide help information
    help_message = (
        "This bot can only process photos. Here are the available commands:\n"
        "/start - Start the bot\n"
        "/help - Display this help message\n"
        "Send me a photo, and I'll process it!"
    )
    await message.answer(help_message)

In [11]:

@dp.message(F.photo)
async def handle_image(message: Message, bot: Bot):
  try:
    logger.info(f"Processing image for user {message.from_user.id}")

    temp_file = f"/tmp/{message.photo[-1].file_id}.jpg"
    new_file = f"/tmp/{message.photo[-1].file_id}_modified.jpg"

    await bot.download(message.photo[-1], destination=temp_file)
    await get_image(temp_file,new_file)

    await message.answer_photo(
        FSInputFile(new_file),
        caption="Here is your processed image")

    log_user(message, temp_file, new_file)
    logger.info(f"Image processed successfully for user {message.from_user.id}")
  except Exception as e:
    logger.error(f"Error handling image for user {message.from_user.id}: {str(e)}")

In [12]:
async def get_image(temp_file, new_file):
    image = Image.open(temp_file)
    img_cv = np.array(image)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        cv2.rectangle(img_cv, (x, y), (x + w, y + h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_cv, 'face', (x, y - 10), font, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
    image = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
    image.save(new_file)
    return image

## Run bot loop

In [13]:
await dp.start_polling(iobot)